In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p nmt
%cd nmt

In [ ]:
!pip3 install SentencePiece

In [ ]:
!git clone https://github.com/ymoslem/MT-Preparation.git

In [ ]:
!cp "/content/drive/MyDrive/en-nl.txt.zip" /content/nmt/

In [ ]:
!unzip en-nl.txt.zip

In [ ]:
# Filter the text
!python MT-Preparation/filtering/filter.py Europarl.en-nl.en Europarl.en-nl.nl en nl

In [ ]:
!python MT-Preparation/subwording/1-train_unigram.py Europarl.en-nl.en-filtered.en Europarl.en-nl.nl-filtered.nl --train_extremely_large_corpus=true

In [ ]:
!python MT-Preparation/subwording/2-subword.py source.model target.model Europarl.en-nl.en-filtered.en Europarl.en-nl.nl-filtered.nl

In [ ]:
!python MT-Preparation/train_dev_split/train_dev_test_split.py 200000 2000 Europarl.en-nl.nl-filtered.nl.subword Europarl.en-nl.en-filtered.en.subword

In [ ]:
!wc -l *.subword.*

   200000 Europarl.en-nl.en-filtered.en.subword.dev
     2000 Europarl.en-nl.en-filtered.en.subword.test
  1757612 Europarl.en-nl.en-filtered.en.subword.train
   200000 Europarl.en-nl.nl-filtered.nl.subword.dev
     2000 Europarl.en-nl.nl-filtered.nl.subword.test
  1757612 Europarl.en-nl.nl-filtered.nl.subword.train
  3919224 total


In [ ]:
!cp -R /content/nmt/ /content/drive/MyDrive/nmt2

cp: cannot stat '/content/nmt/': No such file or directory


In [1]:
#to import all the stuff back
from google.colab import drive
drive.mount('/content/drive')

!cp -R /content/drive/MyDrive/nmt2 /content/nmt/

Mounted at /content/drive


In [3]:
config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: Europarl.en-nl.en-filtered.en.subword.train
        path_tgt: Europarl.en-nl.nl-filtered.nl.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: Europarl.en-nl.en-filtered.en.subword.dev
        path_tgt: Europarl.en-nl.nl-filtered.nl.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.ennl

# Stop training if it does not improve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 200000

# Default: 10000 - Run validation after n steps
valid_steps: 10000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 4000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 2048   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 1024
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps:
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6  # Increased from 6 to 8
dec_layers: 6  # Increased from 6 to 8
heads: 8      # Increased from 8 to 12
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout: [0.1]
attention_dropout: [0.1]

'''

file_path = "/content/nmt/config.yaml"

with open(file_path, "w+") as config_yaml:
    config_yaml.write(config)

In [ ]:
!nproc --all

2


In [ ]:
!pip3 install OpenNMT-py

In [5]:
%cd /content/nmt

/content/nmt


In [6]:
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

2023-10-31 20:14:57.799871: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 20:14:57.799943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 20:14:57.799981: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 20:14:57.808324: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 20:14:59.297716: W tensorflow/compiler/

In [ ]:
!onmt_train -config config.yaml

2023-10-31 20:17:18.262522: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 20:17:18.262573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 20:17:18.262612: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 20:17:18.273806: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 20:17:19.888575: W tensorflow/compiler/

In [ ]:
!head -n 10 Europarl.en-nl.en-filtered.en.subword.test

▁ Y ou ▁actively ▁support ▁globalisation ▁but ▁you ▁scream ▁at ▁the ▁top ▁of ▁your ▁voices ▁when ▁European ▁companies ▁try ▁to ▁adapt ▁to ▁the ▁damaging ▁regulations ▁that ▁you ▁yourselves ▁laid ▁down , ▁as ▁well ▁as ▁to ▁the ▁economic ▁environment ▁that ▁you ▁are ▁imposing ▁on ▁them . ▁Have ▁the ▁courage ▁then , ▁to ▁face ▁up ▁to ▁reality .
▁It ▁would ▁indeed ▁be ▁ideal ▁to ▁create ▁competition ▁structures ▁which ▁include ▁basic ▁principles , ▁such ▁as ▁restricting ▁abusive ▁practices ▁and ▁concentrations , ▁common ▁principles ▁to ▁prohibit ▁unfair ▁competition ▁in ▁the ▁international ▁sphere , ▁and ▁the ▁development ▁of ▁a ▁cooperation ▁instrument ▁based ▁on ▁experience ▁gained ▁in ▁this ▁field .
▁Mr ▁President , ▁the ▁situation ▁in ▁Indonesia ▁is ▁getting ▁worse ▁from ▁week ▁to ▁week , ▁which ▁is ▁not ▁a ▁good ▁omen ▁for ▁the ▁elections ▁on ▁ 7 ▁June .
▁This ▁is ▁an ▁instrument ▁of ▁pressure ▁that ▁is ▁difficult ▁to ▁capture ▁precisely .
▁To ▁these ▁objectives ▁must ▁be ▁added ▁that

In [ ]:
!head -n 10 Europarl.en-nl.nl-filtered.nl.subword.test.desubword

Dames en heren, u werkt actief mee aan de mondialisering, maar begint meteen te roepen wanneer Europese ondernemingen zich proberen aan te passen aan de schadelijke regels die u zelf hebt afgekondigd en aan de economische omgeving die u hen zelf hebt opgelegd.
Dat zou inderdaad de ideale oplossing zijn, waarbij het zaak zou zijn dat er structuren voor de concurrentie werden ontwikkeld op basis van een aantal grondbeginselen zoals de beperking van misbruik en concentratie, gemeenschappelijke principes die concurrentievervalsing op internationaal niveau verbieden, en de ontwikkeling van een instrument ter bevordering van de samenwerking en gebaseerd op de ervaring die op dit gebied is opgedaan.
Voorzitter, de situatie in Indonesië verslechtert van week tot week. Dat is geen goed teken voor de verkiezingen van 7 juni.
Het gaat om een drukmiddel dat moeilijk te vatten is.
Afgezien daarvan hebben wij ook nog het doel van de verbetering van het vrije verkeer van voedingsmiddelen. De opstelli

In [ ]:
!onmt_translate -model models/model.ennl_step_40000.pt -src Europarl.en-nl.en-filtered.en.subword.test -output Europarl.nl.translated -gpu 0 -min_length 1

2023-10-25 16:37:36.904218: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 16:37:36.904288: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 16:37:36.907434: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-25 16:37:37.198594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 16:37:40.404551: W tensorflow/c

In [ ]:
!head -n 5 Europarl.nl.translated

▁In ▁de ▁eerste ▁plaats ▁van ▁de ▁Europese ▁Unie .
▁In ▁de ▁eerste ▁plaats ▁van ▁de ▁Europese ▁Unie .
▁In ▁de ▁eerste ▁plaats ▁van ▁de ▁Europese ▁Unie .
▁In ▁de ▁eerste ▁plaats ▁van ▁de ▁Europese ▁Unie .
▁In ▁de ▁eerste ▁plaats ▁van ▁de ▁Europese ▁Unie .


In [ ]:
!python MT-Preparation/subwording/3-desubword.py target.model Europarl.nl.translated

Done desubwording! Output: Europarl.nl.translated.desubword


In [ ]:
!python MT-Preparation/subwording/3-desubword.py target.model Europarl.en-nl.nl-filtered.nl.subword.test

Done desubwording! Output: Europarl.en-nl.nl-filtered.nl.subword.test.desubword


In [ ]:
# Download BLEU
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-10-25 16:43:41--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2023-10-25 16:43:41 (42.5 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
!pip3 install sacrebleu

In [ ]:
!python compute-bleu.py Europarl.en-nl.nl-filtered.nl.subword.test.desubword Europarl.nl.translated.desubword

Reference 1st sentence: Dames en heren, u werkt actief mee aan de mondialisering, maar begint meteen te roepen wanneer Europese ondernemingen zich proberen aan te passen aan de schadelijke regels die u zelf hebt afgekondigd en aan de economische omgeving die u hen zelf hebt opgelegd.
MTed 1st sentence: In de eerste plaats van de Europese Unie.
BLEU:  0.874830770080218


In [ ]:
!cp -R /content/nmt/ /content/drive/MyDrive/